# Politics in Harvey Mudd

In [1]:
SCHOOL="harveymudd"
SUBJECT="opinions"
START_YEAR=2017
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%pwd

'C:\\Users\\ranar\\ASDRP\\diversity-colleges\\notebooks'

In [3]:
%cd ..

C:\Users\ranar\ASDRP\diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [5]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [6]:
df.shape

(4, 6)

In [7]:
df.head(2)

,title,url,body,year,month,day
0,Opinion: Please Learn to Write a Good Op-ed,https://themuddraker.org/2022/10/01/opinion-pl...,Opinion: Please Learn to Write a Good Op-ed\n,2022,10,1
1,OPINION: HMC Admin Must Do Better in Addressin...,https://themuddraker.org/2021/05/13/opinion-hm...,OPINION: HMC Admin Must Do Better in Addressin...,2021,5,13


In [8]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API_KEY")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [9]:
BIPARTISAN_API_KEY

'gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=='

In [10]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

0.425327


In [11]:
df['year'].unique()

array([2022, 2021, 2020, 2017], dtype=int64)

In [18]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2017, 2023):
    if year == 2018 or year == 2019:
        continue
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2017 ... 1
2017	1	-0.07555	-0.07555
Year: 2020 ... 1
2020	1	0.76118	0.76118
Year: 2021 ... 1
2021	1	0.151384	0.151384
Year: 2022 ... 1
2022	1	0.425327	0.425327


In [19]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)